In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from pprint import pprint
import time

In [10]:
# Initialize Chrome WebDriver using webdriver-manager
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/9y/c00198cn7h9dq4_rd5p9wz0w0000gn/T/ipykernel_78939/1063068479.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [11]:
def get_booking_section(driver):
    return driver \
        .find_element(By.CLASS_NAME, value="resort-book-mobile") \
        .find_elements(By.XPATH, "./div")

def get_booking_section_person(driver):
    return get_booking_section(driver)[0]

def get_booking_section_date(driver):
    return get_booking_section(driver)[2]

def select_date(driver, day, month):
    book_section_date = get_booking_section_date(driver)
    book_section_date.find_element(By.CLASS_NAME, "edit").click() #calendar popup
    #time.sleep(0.2)

    #Month
    month_selector = book_section_date.find_element(By.TAG_NAME, "button")
    month_selector.click()
    #time.sleep(0.2)

    month_cells = book_section_date.find_elements(By.CLASS_NAME, "dp__overlay_cell_pad")
    month_cells_lookup = dict(zip([x.text for x in month_cells], month_cells))
    month_cells_lookup[month].click()
    #time.sleep(0.1)

    #Move the mouse out of the way of popup
    ActionChains(driver)\
        .move_to_element(month_selector)\
        .perform()
    
    #Date
    calendar_section = driver.find_element(By.CLASS_NAME, "dp__calendar")
    date_link = calendar_section.find_element(By.XPATH, f".//div[text()={day}]")
    date_link.click()

def set_persons(driver):
    booking_section_persons = get_booking_section_person(driver)
    booking_section_persons.find_element(By.CLASS_NAME, "edit").click()

    child_exists = len(booking_section_persons.find_elements(By.CLASS_NAME, "child-dropdowns")) > 0
    if not(child_exists):
        booking_section_persons.find_elements(By.CLASS_NAME, "plus")[1].click() #plus child
        booking_section_persons.find_elements(By.CLASS_NAME, "plus")[1].click() #plus child
        booking_section_persons.find_element(By.CLASS_NAME, "edit").click()

def get_room_prices(driver):
    driver.find_element(By.LINK_TEXT, "BOOK NOW").click()
    driver.find_element(By.LINK_TEXT, "Select").click()
    driver.find_element(By.LINK_TEXT, "Accept and Proceed").click()

    #Change room popup
    driver.find_element(By.TAG_NAME, "button").click()
    #time.sleep(0.5)

    prices = {}
    room_popup = driver.find_element(By.ID, "roomtypepopup")
    for room_element in room_popup.find_elements(By.CLASS_NAME, "room"):
        room_type = room_element.find_element(By.CLASS_NAME, "passengers").text
        price = room_element.find_element(By.CLASS_NAME, "price").text
        prices[room_type] = price

        if len(room_type) == 0:
            print("Empty Room Details...")
            print(room_popup.get_attribute('outerHTML'))
    
    #close popup
    driver.find_element(By.CLASS_NAME, "popup-close").click()

    #Go back to starting point
    driver.back()
    driver.back()

    return prices

In [13]:
#driver = webdriver.Chrome()
driver.get("https://www.markwarner.co.uk/")
driver.implicitly_wait(0.5)

#Accept cookie
cookie_accept_button = driver.find_elements(by=By.ID, value="CybotCookiebotDialogBodyLevelButtonAccept")
if len(cookie_accept_button) > 0:
    cookie_accept_button[0].click()

locations = {
    "Paleros": "https://www.markwarner.co.uk/sun-holidays/greece/paleros-beach-resort/",
    "Phokaia": "https://www.markwarner.co.uk/sun-holidays/turkey/phokaia-beach-resort/",
    "Aeolian Village": "https://www.markwarner.co.uk/sun-holidays/greece/aeolian-village/",
    "Kamari": "https://www.markwarner.co.uk/sun-holidays/greece/kamari-beach-resort/",
    "Lemnos": "https://www.markwarner.co.uk/sun-holidays/greece/lemnos-beach-resort/",
}

all_prices = []
for location, url in locations.items():
    print(f"Processing: {location} @ {url}...")
    driver.get(url)

    for month in ["Jul", "Aug"]:

        set_persons(driver)

        #Find all the available dates
        book_section_date = get_booking_section_date(driver)
        book_section_date.find_element(By.CLASS_NAME, "edit").click()
        book_section_date.find_element(By.TAG_NAME, "button").click()

        #time.sleep(0.5)
        
        month_cells = book_section_date.find_elements(By.CLASS_NAME, "dp__overlay_cell_pad")
        month_cells_lookup = dict(zip([x.text for x in month_cells], month_cells))
        month_cells_lookup[month].click()

        #time.sleep(0.1) #odd race condidtion to read calendar days

        #Move the mouse out of the way of popup
        ActionChains(driver)\
            .move_by_offset(10, 0)\
            .perform()

        available_date_cells = book_section_date.find_elements(By.CLASS_NAME, "dp__date_hover")
        active_date_cell = book_section_date.find_elements(By.CLASS_NAME, "dp__active_date")
        combined_dates = [x.text for x in (available_date_cells + active_date_cell)]

        book_section_date.find_element(By.CLASS_NAME, "edit").click()

        for day in combined_dates:
            #may have gone stale
            set_persons(driver)
            select_date(driver, day, month)
            price = driver.find_element(By.CLASS_NAME, "book").find_element(By.TAG_NAME, "b").text
            print("Headline", {"Month": month, "Day":day, "Price":price})

            prices = get_room_prices(driver)
            prices.update(Month=month, Day=day, Resort=location)
            all_prices.append(prices)

Processing: Paleros @ https://www.markwarner.co.uk/sun-holidays/greece/paleros-beach-resort/...
Headline {'Month': 'Jul', 'Day': '7', 'Price': '£6,336.00'}
Headline {'Month': 'Jul', 'Day': '14', 'Price': '£6,396.00'}
Headline {'Month': 'Jul', 'Day': '21', 'Price': '£6,656.00'}
Headline {'Month': 'Jul', 'Day': '28', 'Price': '£6,656.00'}
Headline {'Month': 'Aug', 'Day': '4', 'Price': '£6,656.00'}
Headline {'Month': 'Aug', 'Day': '7', 'Price': '£7,446.00'}
Headline {'Month': 'Aug', 'Day': '11', 'Price': '£6,656.00'}
Headline {'Month': 'Aug', 'Day': '18', 'Price': '£6,596.00'}
Headline {'Month': 'Aug', 'Day': '25', 'Price': '£5,546.00'}
Processing: Phokaia @ https://www.markwarner.co.uk/sun-holidays/turkey/phokaia-beach-resort/...
Headline {'Month': 'Jul', 'Day': '2', 'Price': '£4,656.00'}
Headline {'Month': 'Jul', 'Day': '9', 'Price': '£5,771.00'}
Headline {'Month': 'Jul', 'Day': '16', 'Price': '£5,621.00'}
Headline {'Month': 'Jul', 'Day': '23', 'Price': '£6,256.00'}
Headline {'Month': '

In [19]:
import pandas as pd

df = pd.DataFrame(all_prices).set_index(["Resort","Month","Day"])
df = df[df.columns.sort_values()]
df

Accessible twin/double Family rooms  \
Resort          Month Day                                       
Paleros         Jul   7                      NaN          NaN   
                      14                     NaN        £6446   
                      21                     NaN        £6676   
                      28                     NaN        £6676   
                Aug   4                      NaN        £6676   
                      7                      NaN        £7446   
                      11                     NaN        £6676   
                      18                     NaN        £6596   
                      25                     NaN        £5546   
Phokaia         Jul   2                      NaN          NaN   
                      9                      NaN          NaN   
                      16                     NaN          NaN   
                      23                     NaN          NaN   
                      30                     NaN          NaN   
                Aug   6                      NaN          NaN   
                      13                     NaN          NaN   
                      20                     NaN          NaN   
                      27                     NaN          NaN   
Aeolian Village Jul   3                      NaN          NaN   
                      10                   £3198          NaN   
                      17                   £3008          NaN   
                      24                     NaN          NaN   
                      31                     NaN          NaN   
                Aug   7                      NaN          NaN   
                      14                     NaN          NaN   
                      21                     NaN          NaN   
                      28                     NaN          NaN   
Kamari          Jul   6                      NaN        £5706   
                      13                     NaN        £5706   
                      20                     NaN        £5646   
                      27                     NaN        £6026   
                Aug   3                      NaN          NaN   
                      10                     NaN          NaN   
                      17                     NaN        £6026   
                      24                     NaN        £5606   
                      31                     NaN        £3106   
Lemnos          Jul   5                      NaN          NaN   
                      12                     NaN          NaN   
                      19                     NaN          NaN   
                      26                     NaN          NaN   
                Aug   2                      NaN          NaN   
                      9                      NaN          NaN   
                      16                     NaN          NaN   
                      23                     NaN          NaN   
                      30                     NaN          NaN   

                          Family rooms (ensuite shower)  \
Resort          Month Day                                 
Paleros         Jul   7                             NaN   
                      14                            NaN   
                      21                            NaN   
                      28                            NaN   
                Aug   4                             NaN   
                      7                             NaN   
                      11                            NaN   
                      18                            NaN   
                      25                            NaN   
Phokaia         Jul   2                           £4656   
                      9                           £5771   
                      16                          £5621   
                      23                          £6501   
                      30                          £6541   
                Aug   6   

In [20]:
df.to_csv("./out.csv")